In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing as preprocessing
from sklearn import model_selection, svm
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
import data_process_funcs as dpf
import ml_funcs as mlf

In [6]:
df_8q_R = dpf.get_expanded_df('Refreshed_Simulation',8)


In [ ]:
df_8q_Rp = mlf.apply_preprosessing(df_8q_R)
model1 = SVC(kernel='linear')
print("8 qubits Hardware:")
fitted_model,score,cv_score = mlf.std_split_fit_and_scores(df_8q_Rp,model1)

8 qubits Hardware:
Cross-validation accuracy:  [0.98958333 1.         0.98958333 0.98958333 1.        ]


In [8]:
filename, fields = mlf.create_ml_results_csv('SVM')
print("Filename:", filename)
print("Fields:", fields)

Filename: ../ML_Results/SVM_results.csv
Fields: ['nr_qubits', 'machines', 'tr&v exp_type', 'tr&v circuits', 'test exp_type', 'test circuits', 'preprocess settings', 'kernal', 'param settings', 'accuracy', 'cv_1', 'cv_2', 'cv_3', 'cv_4', 'cv_5']


In [9]:
machine_bin = mlf.get_machine_binary(['torino', 'brisbane'])
print("Machine binary:", machine_bin)

Machine binary: 1100


In [10]:
# circuit_bin = mlf.get_circuit_binary(['1','2','3'])
circuit_bin = mlf.get_circuit_binary_from_df(df_8q_Rp)
print("Circuit binary:", circuit_bin)

Circuit binary: 111


In [11]:
general_fields = mlf.get_general_fields(8, machine_bin, 'Hardware', circuit_bin, 'Hardware', circuit_bin, 0)
print("General fields:", general_fields)

General fields: {'nr_qubits': 8, 'machines': '1100', 'tr&v exp_type': 'Hardware', 'tr&v circuits': '111', 'test exp_type': 'Hardware', 'test circuits': '111', 'preprocess settings': 0}


In [12]:
ml_fields = mlf.get_ml_fields('SVM', model1.get_params())
print("ML fields:", ml_fields)

ML fields: {'kernal': 'linear', 'param settings': 0}


In [ ]:
# result_fields = get_results_fields(accuracy_score(target_test, target_pred), scores)
result_fields = mlf.get_results_fields(score, cv_score)
print("Result fields:", result_fields)

Result fields: {'accuracy': 0.9916666666666667, 'cv_1': np.float64(0.9895833333333334), 'cv_2': np.float64(1.0), 'cv_3': np.float64(0.9895833333333334), 'cv_4': np.float64(0.9895833333333334), 'cv_5': np.float64(1.0)}


In [15]:
mlf.ml_results_to_csv(general_fields, ml_fields, result_fields, filename, fields)

## Jenna's code explanation

In [3]:
#load data
nr_qubits =4
df_4q_R = dpf.get_expanded_df('Refreshed_Simulation',nr_qubits)
#df_4q_H = dpf.get_expanded_df('Hardware',nr_qubits)
#df_4q_S = dpf.get_expanded_df('Simulation',nr_qubits)

In [4]:
#initiate ml alg
model1 = SVC(kernel='linear')

In [ ]:
#make the different combinations

#the input to this function should be an array of dataframes that you want 
# to make different combinations from.
#the output will be an array of the dataframes in different orders

#eg. mlf.generate_combos([df1,df2,df3])

#output will be:
#[[df1, df2, df3],
#[df2, df1, df3],
# df3, df1, df2]]

#if you make include_combined=True:
#mlf.generate_combos([df1,df2,df3],True)

#output will be:
#[[df1, df2, df3,pd.concat(df2,df3)],
#[df2, df1, df3,pd.concat(df1,df3)],
# df3, df1, df2,pd.concat(df1,df2)]]


In [5]:
#apply preprocessing
df_4q_Ri = mlf.features_to_int(df_4q_R)
#to_int is the most important here, the other parts of preprocessing arent
#neccessary yet

#you can use any array of dfs that all have the smae nr_qubits
# here I use split_into_circuits to get 3 dfs, one for each circuit_type

circuits = mlf.split_into_circuits(df_4q_Ri) 
#result is an array of dfs in the form [df_c1,df_c2,df_c3]
combos = mlf.generate_combos(circuits,True)


In [6]:

#test on the first combo:
combo_1 = combos[0]
train_label = "c1"
test_labels = ["test on c2","test on c3","test on c2,3"]
tr_val_df = combo_1[0]
test_dfs = combo_1[1:]

#apply preprocessing
tr_val_dfp = mlf.apply_preprosessing(tr_val_df)
test_dfps = []
for dfp in test_dfs:
    dfp = mlf.apply_preprosessing(dfp)
    test_dfps.append(dfp)


scores,labels= mlf.get_accuracies_for_comparison(
model1, tr_val_dfp,train_label, test_dfps,test_labels, True)

Trained on  c1
test on  self_score : 1.0
test on  test on c2 : 0.46
test on  test on c3 : 0.76
test on  test on c2,3 : 0.5375


In [7]:
print(scores)

[1.0, 0.46, 0.76, 0.5375]


In [9]:
print(labels)

['self_score', 'test on c2', 'test on c3', 'test on c2,3']


In [ ]:
#test on the 2nd combo:
combo_1 = combos[1]
train_label = "c2"
test_labels = ["test on c1","test on c3","test on c1,3"]
tr_val_df = combo_1[0]
test_dfs = combo_1[1:]

#apply preprocessing
tr_val_dfp = mlf.apply_preprosessing(tr_val_df)
test_dfps = []
for dfp in test_dfs:
    dfp = mlf.apply_preprosessing(dfp)
    test_dfps.append(dfp)


scores,labels= mlf.get_accuracies_for_comparison(
model1, tr_val_dfp,train_label, test_dfps,test_labels, True)

In [5]:
df_4q_Rp = mlf.apply_preprosessing(df_4q_R)

In [6]:
df_4q_Rp.head()

,circuit_type,backend,totalError,0001,0010,0011,0100,0101,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111
0,0.333333,1,0.011963,0.081633,0.387755,0.0,0.285714,0.0,0.000,0.0,0.244898,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.333333,1,0.012207,0.100000,0.360000,0.0,0.280000,0.0,0.000,0.0,0.260000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.333333,1,0.010010,0.121951,0.341463,0.0,0.365854,0.0,0.000,0.0,0.170732,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.333333,1,0.011475,0.106383,0.404255,0.0,0.276596,0.0,0.000,0.0,0.212766,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.333333,1,0.009766,0.075000,0.375000,0.0,0.350000,0.0,0.025,0.0,0.175000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
